In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# CDF Response Time Analysis Example 1
# Demonstrates cumulative distribution function analysis for a closed network

model = Network('model')

# Create closed queueing network with Delay and Queue nodes
node = [None] * 2  # using 0-based indexing
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue2', SchedStrategy.PS)

# Single closed class with 1 job starting at Delay node
jobclass = [None] * 1  # using 0-based indexing
jobclass[0] = ClosedClass(model, 'Class1', 1, node[0], 0)

# Service processes - match MATLAB exactly
servProc1 = Exp(1/0.1)  # rate = 1/0.1 = 10
node[0].set_service(jobclass[0], servProc1)

servProc2 = Erlang.fit_mean_and_scv(1, 1/3)  # mean=1, SCV=1/3
node[1].set_service(jobclass[0], servProc2)

# Routing matrix - circular routing between the two nodes
P = model.init_routing_matrix()
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)
P.set(jobclass[0], jobclass[0], node[1], node[0], 1.0)
model.link(P)

In [3]:
# Solve with JMT solver - aligned with JAR test scenario
solver = JMT(model, seed=23000, samples=10000)  # Aligned with JAR test: seed=23000, samples=10000
FC = solver.cdf_respt()

AvgRespTfromCDFSim = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))
PowerMoment2_R = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))
Variance_R = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))
SqCoeffOfVariationRespTfromCDFSim = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))

for i in range(model.get_number_of_stations()):
    for c in range(model.get_number_of_classes()):
        if FC[i][c] is not None and len(FC[i][c]) > 1:
            # Calculate mean from CDF
            diffs = np.diff(FC[i][c][:, 0])
            values = FC[i][c][1:, 1]
            AvgRespTfromCDFSim[i, c] = np.sum(diffs * values)
            
            # Calculate second moment and variance
            PowerMoment2_R[i, c] = np.sum(diffs * (values ** 2))
            Variance_R[i, c] = PowerMoment2_R[i, c] - AvgRespTfromCDFSim[i, c] ** 2
            SqCoeffOfVariationRespTfromCDFSim[i, c] = Variance_R[i, c] / (AvgRespTfromCDFSim[i, c] ** 2)

# Solve with Fluid solver - aligned with JAR test scenario (default options)
solver = FLD(model)  # Using default options like JAR test
FC = solver.cdf_respt()

AvgRespTfromCDFFluid = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))
SqCoeffOfVariationRespTfromCDFFluid = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))

for i in range(model.get_number_of_stations()):
    for c in range(model.get_number_of_classes()):
        if FC[i][c] is not None and len(FC[i][c]) > 1:
            # Calculate mean from CDF  
            diffs = np.diff(FC[i][c][:, 0])
            values = FC[i][c][1:, 1]
            AvgRespTfromCDFFluid[i, c] = np.sum(diffs * values)
            
            # Calculate second moment and variance
            PowerMoment2_R_fluid = np.sum(diffs * (values ** 2))
            Variance_R_fluid = PowerMoment2_R_fluid - AvgRespTfromCDFFluid[i, c] ** 2
            SqCoeffOfVariationRespTfromCDFFluid[i, c] = Variance_R_fluid / (AvgRespTfromCDFFluid[i, c] ** 2)

# Theoretical values from service processes
AvgRespTfromTheory = np.array([servProc1.get_mean(), servProc2.get_mean()])
SqCoeffOfVariationRespTfromTheory = np.array([servProc1.get_scv(), servProc2.get_scv()])

print('AvgRespTfromTheory =')
print(AvgRespTfromTheory)
print('AvgRespTfromCDFSim =')
print(AvgRespTfromCDFSim)
print('AvgRespTfromCDFFluid =')
print(AvgRespTfromCDFFluid)
print('SqCoeffOfVariationRespTfromTheory =')
print(SqCoeffOfVariationRespTfromTheory)
print('SqCoeffOfVariationRespTfromCDFSim =')
print(SqCoeffOfVariationRespTfromCDFSim)
print('SqCoeffOfVariationRespTfromCDFFluid =')
print(SqCoeffOfVariationRespTfromCDFFluid)

AvgRespTfromTheory =
[0.1 1. ]
AvgRespTfromCDFSim =
[[0.10401799]
 [1.03066911]]
AvgRespTfromCDFFluid =
[[0.10320903]
 [1.03209029]]
SqCoeffOfVariationRespTfromTheory =
[1.         0.33333333]
SqCoeffOfVariationRespTfromCDFSim =
[[1.10596686]
 [1.10596686]]
SqCoeffOfVariationRespTfromCDFFluid =
[[1.10596686]
 [1.10596686]]
